### Uploading DataFrame to SQL DB in AWS
We already scraped the Car Data, now we are going to clean it and make some adjustments to upload it to a Cloud server SQL

Steps:
* import DF from scrapper
* clean data
* Verify columns names
* Upload to DB

In [6]:
import pandas as pd
import math
import psycopg2 as ps
import numpy as np
from dotenv import load_dotenv
import os

In [188]:
df = pd.read_csv('autos_mercadolibre.csv')
df2 = pd.read_csv('last_scrap.csv')

In [189]:
df = pd.concat([df2, df2], ignore_index=True)

In [190]:
df = df.rename(columns={df.columns[9]: 'Tipo de carr'})
df

,Unnamed: 0,Marca,Modelo,AÃ±o,Color,Tipo de combustible,Puertas,TransmisiÃ³n,Motor,Tipo de carr,KilÃ³metros,Location,Price,Link,Año,Transmisión,Tipo de carrocería,Kilómetros
0,0,Alfa Romeo,Mito,2015.0,Blanco,Nafta,3,AutomÃ¡tica,1.4,Hatchback,47800 km,Ramos MejÃ­a - La Matanza - Bs.As. G.B.A. Oeste,3800000,https://auto.mercadolibre.com.ar/MLA-113834276...,NaN,NaN,NaN,NaN
1,1,Audi,A5 Sportback,NaN,Gris,Nafta,5,NaN,3.2,NaN,NaN,Villa Martelli - Vicente López - Bs.As. G.B.A....,23500,https://auto.mercadolibre.com.ar/MLA-113760527...,2011.0,Automática,Hatchback,110000 km
2,2,Audi,A6,2013.0,Gris,Nafta,4,AutomÃ¡tica,2.8,SedÃ¡n,210000 km,Tigre - Bs.As. G.B.A. Norte,27000,https://auto.mercadolibre.com.ar/MLA-113750858...,NaN,NaN,NaN,NaN
3,3,Audi,A4,2007.0,NaN,DiÃ©sel,4,Manual,2.0,SedÃ¡n,203000 km,TapalquÃ© - Buenos Aires Interior,1700000,https://auto.mercadolibre.com.ar/MLA-113044459...,NaN,NaN,NaN,NaN
4,4,Audi,A4,2012.0,Blanco,Nafta,4,Manual,1.8,SedÃ¡n,110000 km,Lomas de Zamora - Lomas de Zamora - Bs.As. G.B...,16000,https://auto.mercadolibre.com.ar/MLA-113745157...,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38643,19319,Volvo,XC40,2019.0,Negro,Nafta,4,AutomÃ¡tica,NaN,SUV,5000 km,Belgrano - Capital Federal - Capital Federal,99000,https://auto.mercadolibre.com.ar/MLA-113029995...,NaN,NaN,NaN,NaN
38644,19320,Volvo,S 122,1963.0,NaN,Nafta,2,NaN,NaN,NaN,111111 km,La Matanza - Bs.As. G.B.A. Oeste,5000,https://auto.mercadolibre.com.ar/MLA-113208746...,NaN,NaN,NaN,NaN
38645,19321,Volvo,XC60,2011.0,Azul,Nafta,5,AutomÃ¡tica,2.0,SUV,135000 km,Centro - Rosario - Santa Fe,14000,https://auto.mercadolibre.com.ar/MLA-110875003...,NaN,NaN,NaN,NaN
38646,19322,Volvo,S40,2011.0,Blanco,Nafta,4,AutomÃ¡tica,NaN,SedÃ¡n,68000 km,Caballito - Capital Federal - Capital Federal,2400000,https://auto.mercadolibre.com.ar/MLA-112101811...,NaN,NaN,NaN,NaN


We see that there is a problem with the column names which endes up in NaN values were in reallity there are 3 columns that are the same (This is because of character encoding)
This is why we rename all columns into english name to prevent compatibility issues

In [191]:
new_df = pd.DataFrame()

In [211]:
new_df['id'] = df.index
new_df['brand'] = df['Marca']
new_df['model'] = df['Modelo']
new_df['colour'] = df['Color']
new_df['fuel'] = df['Tipo de combustible']
new_df['doors'] = df['Puertas']
new_df['engine'] = df['Motor']
new_df['location'] = df['Location']
new_df['price'] = df['Price']
new_df['year'] = df['Año'].fillna(df['AÃ±o'])
new_df['transmision'] = df['Transmisión'].fillna(df['TransmisiÃ³n'])
new_df['km'] = df['Kilómetros'].fillna(df['KilÃ³metros'])
new_df['type'] = df['Tipo de carrocería'].fillna(df['Tipo de carr'])
new_df['url'] = df['Link']

In [212]:
new_df.isnull().any()

id             False
brand          False
model          False
colour          True
fuel           False
doors          False
engine          True
location       False
price          False
year           False
transmision     True
km             False
type            True
url            False
dtype: bool

As we see there are some missing values, but at least every car has the main features

We also check for duplicates in the URL

In [213]:
new_df.duplicated(subset="id").any()

False

In [214]:
new_df = new_df.drop_duplicates(subset="id")

### As we see we also have a problem with encoding, 2 values ended up beeing 4
#### Sure there is a library to handle encoding, I triend .encode and .decode but didnt work. So I decided to replace it myself

In [215]:
new_df["transmision"].unique()

array(['AutomÃ¡tica', 'Automática', 'Manual', nan,
       'Automática secuencial', 'AutomÃ¡tica secuencial',
       'SemiautomÃ¡tica'], dtype=object)

In [216]:
new_df["transmision"] = new_df["transmision"].str.replace("Ã¡", "á",regex=False)
new_df["fuel"] = new_df["fuel"].str.replace("Ã©", "é",regex=False)
new_df["fuel"] = new_df["fuel"].str.replace("\\", "í",regex=False)


 Also I found this, to normalize and get rid of accents

In [217]:
# From https://stackoverflow.com/a/518232
import unicodedata
def strip_accents(s):
   return ''.join(c for c in unicodedata.normalize('NFD', s)
                  if unicodedata.category(c) != 'Mn')

In [218]:
def series_stripper(serie):
    return serie.apply(lambda x:strip_accents(x))

In [219]:
new_df['fuel'] = series_stripper(new_df['fuel'])
new_df['location'] = series_stripper(new_df['location'])
new_df['transmision'] = series_stripper(new_df['transmision'].fillna(''))
new_df['colour'] = series_stripper(new_df['colour'].fillna('')).str.replace("A³","o",regex=False)
new_df['type'] = series_stripper(new_df['type'].fillna(''))
new_df["brand"] = new_df["brand"].str.replace("Ã«","e",regex=False).str.replace("ë","e",regex=False).str.replace("ÃƒÂ«","e",regex=False)
new_df["fuel"] = new_df["fuel"].str.replace("A\xad","i",regex=False)
new_df["km"] = new_df["km"].str.replace(" km","",regex=False).astype(np.int64)
new_df["type"] = new_df["type"].str.replace("A¡","a",regex=False).str.replace("A³","o",regex=False).str.replace("A©","e",regex=False)
new_df["location"] = new_df["location"].str.replace("A¡","a",regex=False).str.replace("A³","o",regex=False).str.replace("A©","e",regex=False).str.replace("A\xad","i",regex=False).str.replace("A±","n",regex=False).str.replace("Aº","u",regex=False).str.replace("A¼","u",regex=False)
new_df["year"] = new_df["year"].astype(np.int64)
new_df[new_df["km"]>1000000]=1000000 #This prevents SQL to get int out of range
#Index(['brand', 'model', 'colour', 'fuel', 'doors', 'engine', 'location',
 #      'price', 'year', 'transmision', 'km', 'type', 'url'],
  #    dtype='object')

In [220]:
unique = []
unique = new_df.apply(lambda x : x.unique())
unique["brand"]

array(['Alfa Romeo', 'Audi', 'BMW', 'Chery', 1000000, 'Chevrolet',
       'Peugeot', 'Ford', 'Chevrolet Classic', 'Chrysler', 'Jeep',
       'CRYSLER TONW & COUNTRY', 'Chrysler stratus lx', 'Mercedes-Benz',
       'Citroen', 'Citroen c4 feel pack', 'CitroÂ´n', 'Citroen C4 CACTUS',
       'Renault', 'Dodge', 'RAM', 'Volkswagen', 'Dogde', 'DS', 'Toyota',
       'Fiat', 'Honda', 'Honda legend', 'Hyundai', 'Hyunday', 'Isuzu',
       'JMC', 'izuzu', 'Jeep ika continental', 'IKA', 'Ika',
       'Ika Renault', 'Suzuki', 'Escucho oferta Jeep', 'Aro', 'Kia',
       'KIA SOULL', 'Land Rover', 'Range Rover', 'Rover', 'Mazda',
       'mercedes', 'MERCEDEZ BENZ', 'Mercedes Benz', 'Mercedez Benz',
       'Sprinter', 'Mercedes Benz C250 B', 'Mercedes Benz 1318',
       'Shineray', 'Mini', 'Mini Cooper', 'Iveco', 'Mini cooper JCW',
       'Mitsubishi', 'Nissan', 'NISSAN NX 2000', 'Nissan datsun',
       'Peugeot honda nissan', 'Porsche', 'Smart', 'Subaru',
       'Suzuki Nakai', 'SUZUKY FUN', 'Toyota 

### Now we got a much cleaner DataFrame

In [221]:
new_df.columns

Index(['id', 'brand', 'model', 'colour', 'fuel', 'doors', 'engine', 'location',
       'price', 'year', 'transmision', 'km', 'type', 'url'],
      dtype='object')

##### We check the types of our DF and translate to SQL types

In [222]:
new_df.dtypes

id              int64
brand          object
model          object
colour         object
fuel           object
doors           int64
engine         object
location       object
price           int64
year            int64
transmision    object
km              int64
type           object
url            object
dtype: object

In [223]:
replacements = {
    'object':'varchar',
    'int64':'int',
    'float64':'float'
}

In [224]:
col_str = ", ".join("{} {}".format(n, d) for (n, d) in zip(new_df.columns[1:], new_df.dtypes[1:].replace(replacements)))
"id int PRIMARY KEY, " + col_str

'id int PRIMARY KEY, brand varchar, model varchar, colour varchar, fuel varchar, doors int, engine varchar, location varchar, price int, year int, transmision varchar, km int, type varchar, url varchar'

##### DB connection

In [27]:
load_dotenv()
host_name = os.environ.get('SQL_HOST_NAME')
dbname = "postgres"
port = "5432"
username = os.environ.get('SQL_USER_NAME')
password =  os.environ.get('SQL_PASSWORD')
conn = None

In [28]:
try:
    conn = ps.connect(host=host_name, database=dbname, user=username, password=password, port=port)
except ps.OperationalError as e:
    raise e
else:
    print("Connected!")

Connected!


In [106]:
create_table = (f"""CREATE TABLE IF NOT EXISTS cars ({col_str})""")

In [80]:
conn.autocommit = True
curr = conn.cursor()

In [81]:
curr.execute(create_table)

In [225]:
new_df.to_csv("tosqldb.csv", header=new_df.columns, index=False, encoding='utf-8')
cars_data = cars_data.drop(cars_data[cars_data["id"]>40000].index)
csv = open("tosqldb.csv", encoding="utf-8")

Save and open to insert into DB

In [84]:
COPY_QUERY = """
COPY cars FROM STDIN WITH CSV HEADER DELIMITER AS ','
"""
curr.copy_expert(sql=COPY_QUERY, file=csv)

In [ ]:
curr.execute("SELECT * FROM cars WHERE year=2019")
curr.fetchone()

In [ ]:
'AutomÃ¡tica'.encode('latin-1').decode('utf-8')